# *N*-gram Language Models

In this notebook, we'll be building bigram *n*-gram language models from scratch. The first part of building a language model is collecting counts from corpora. To avoid problems with unknown words, we'll turn all *hapax legomena* (words that appear once) to UNK, and we'll also lower case everything and add `<s>` (start) and `</s>` (end) symbols at the beginning and end of each sentence. For bigrams, we are using dictionaries of dictionaries with the strings as keys, which is a convenient though not particularly memory efficient way to represent things. We will use the unigram counts later for doing smoothing.

In [ ]:
from collections import defaultdict

def add_unks(word,unigram_counts):
    if word in unigram_counts:
        return word
    else:
        return "UNK"

def convert_sentence_train(sentence,unigram_counts):
    return ["<s>"] + [add_unks(token.lower(),unigram_counts) for token in sentence] + ["</s>"]

def get_counts(sentences):
    bigram_counts = defaultdict(dict)
    unigram_counts = {}
    unigram_counts["<s>"] = 0
    # collect initial unigram statistics
    for sentence in sentences:
        for word in sentence:
            word = word.lower()
            unigram_counts[word] = unigram_counts.get(word,0) + 1
        unigram_counts["<s>"] += 1
    unigram_counts["</s>"] = unigram_counts["<s>"]
    # convert hapax legomena to UNK
    unk_words = []
    for word in unigram_counts:
        if unigram_counts[word] == 1:
            unk_words.append(word)
    unigram_counts["UNK"] = len(unk_words)
    for word in unk_words:
        del unigram_counts[word]
    # collect bigram counts
    for sentence in sentences:
        sentence = convert_sentence_train(sentence, unigram_counts)
        for i in range(len(sentence) - 1):
            bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
            
    token_count = float(sum(unigram_counts.values()))
    return unigram_counts, bigram_counts, token_count

Once we have counts, we can use them to generate sentences. Here we use [numpy.random.choice](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.random.choice.html), which allows us to randomly choose from a list based on a corresponding list of probabilities, which we calculate by normalizing the raw counts. We start with &lt;s&gt;, and generate the next word given the bigram counts which begin with &lt;s&gt;, and then use that word to generate the next word, etc. It stops when it generates an &lt;/s&gt;. We return a string with some fixes to make the sentence look a proper sentence.

In [ ]:
from numpy.random import choice 

def generate_sentence(bigram_counts):
    sentence = ["<s>"]
    while sentence[-1] != "</s>":
        prev_token_counts = bigram_counts[sentence[-1]]
        bigram_probabilities = []
        total_counts = float(sum(prev_token_counts.values()))
        for token in prev_token_counts:
            bigram_probabilities.append(prev_token_counts[token]/total_counts)
        sentence.append(choice(prev_token_counts.keys(), p=bigram_probabilities))
    return " ".join([sentence[1].title()] + sentence[2:-1]).replace(" ,",",").replace(" .", ".")
        
            

Now let's try out our *n*-gram driven sentence generator on samples from two corpora: the Penn Treebank, and some out-of-copyright English literature from Project Gutenberg:

In [ ]:
from nltk.corpus import gutenberg, treebank
gutenberg_unigrams, gutenberg_bigrams, gutenberg_token_count = get_counts(gutenberg.sents())
print "Gutenberg"
for i in range(1,6):
    print "Sentence %d" % i 
    print generate_sentence(gutenberg_bigrams)
    
treebank_unigrams, treebank_bigrams, treebank_token_count = get_counts(treebank.sents())
print "Treebank"
for i in range(1,6):
    print "Sentence %d" % i 
    print generate_sentence(treebank_bigrams)

Generally, we can see some local coherence but most of these sentences are complete nonsense. Across the two corpora, the sentences are noticeably different, it's very obvious that the model from Project Gutenberg is trained on literature, whereas the Penn Treebank data is financial. For the latter, there are some strange tokens (those starting with \*) we should probably have filtered out; there are some UNK tokens, which is to be expected: there are a lot of words that appear only once which makes it a fairly common word type.

Using language models to generate sentences is fun but not very useful. A more practical application is the ability to assign a probability to a sentence. In order to do that for anything but toy examples, however, we will need to smooth our models so it doesn't assign a zero probability to the sentence whenever it sees a bigram. Here, we'll test two fairly simple smoothing techniques, add-*k* smoothing and interpolated smoothing. For the latter, we'll control the trade-off between bigrams and unigrams with a simple parameter (bigram_lambda). In both cases, we will calculate the log probability, to avoid working with very small numbers. The functions below give the probability for a single word at index i in a sentence.

In [ ]:
import math

def get_log_prob_addk(sentence,i, unigram_counts,bigram_counts, k):
    return math.log((bigram_counts[sentence[i-1]].get(sentence[i],0) + k)/ \
                    (unigram_counts[sentence[i-1]] + k*len(unigram_counts)))


def get_log_prob_interp(sentence,i, unigram_counts,bigram_counts,token_count, bigram_lambda):    
    return math.log(bigram_lambda*bigram_counts[sentence[i-1]].get(sentence[i],0)/float(unigram_counts[sentence[i-1]]) + \
                    (1 - bigram_lambda)*unigram_counts[sentence[i]]/token_count)


Extending this to calculate the probability of an entire sentence is trivial, except we need to be careful to deal with words we've never seen before, which should be converted to UNK.

In [ ]:
def convert_sentence_test(sentence,unigram_counts):
    return ["<s>"] + [add_unks(word.lower(),unigram_counts) for word in sentence] + ["</s>"]

def get_sent_log_prob_addk(sentence, unigram_counts, bigram_counts, token_count, k):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_addk(sentence,i, unigram_counts,bigram_counts,k) for i in range(1,len(sentence))])

def get_sent_log_prob_interp(sentence, unigram_counts, bigram_counts, token_count, bigram_lambda):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_interp(sentence,i, unigram_counts,bigram_counts,token_count,bigram_lambda) for i in range(1,len(sentence))])

sentence = "revenue increased last quarter .".split()
print get_sent_log_prob_addk(sentence, gutenberg_unigrams, gutenberg_bigrams, gutenberg_token_count, 0.05)
print get_sent_log_prob_interp(sentence, gutenberg_unigrams, gutenberg_bigrams, gutenberg_token_count,0.95)
print get_sent_log_prob_addk(sentence, treebank_unigrams, treebank_bigrams, treebank_token_count, 0.05)
print get_sent_log_prob_interp(sentence, treebank_unigrams, treebank_bigrams, treebank_token_count,0.95)

The output for our sample sentence looks reasonable, in particular using the Treebank model results in a noticeably higher probability, which is what we'd expect given the input sentence. The differences in probability between the different smoothing techniques is more modest (though keep in mind this is a logrithmic scale). Now, let's use perplexity to evaluate different smoothing techniques at the level of the corpus. For this, we'll use the Brown corpus again, dividing it up randomly into a training set and a test set based on an 80/20 split.

In [ ]:
from nltk.corpus import brown
from random import shuffle

sents = list(brown.sents())
shuffle(sents)
cutoff = int(0.8*len(sents))
training_set = sents[:cutoff]
test_set = [[word.lower() for word in sent] for sent in sents[cutoff:]]

brown_unigrams, brown_bigrams, brown_token_count = get_counts(training_set)

Since our probabilities are in log space, we will calculate perplexity in log space as well, then take the exponential at the end

$PP(W) = \sqrt[m]{\frac{1}{P(W)}}$

$\log{PP(W)} = -\frac{1}{m} \log{P(W)}$

In [ ]:
def calculate_perplexity(sentences,unigram_counts,bigram_counts, token_count, smoothing_function, parameter):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 1 # have to consider the end token
        total_log_prob += smoothing_function(sentence,unigram_counts,bigram_counts,token_count, parameter)
    return math.exp(-total_log_prob/test_token_count)

                

Let's see how our two smoothing techniques do with a range of possible parameter values 

In [ ]:
print "add k"
for k in [0.0001,0.001,0.01, 0.05,0.2,1.0]:
    print k
    print calculate_perplexity(test_set,brown_unigrams,brown_bigrams,brown_token_count,get_sent_log_prob_addk,k)
print "interpolation"
for bigram_lambda in [0.99,0.95,0.75,0.5,0.25,0.001]:
    print bigram_lambda
    print calculate_perplexity(test_set,brown_unigrams,brown_bigrams,brown_token_count,get_sent_log_prob_interp,bigram_lambda)   

Our results indicate that, with regards to perplexity, interpolation is generally better than add k. Very low (though not too low) k is preferred for add k, wheres our best bigram lambda is in the middle of the range, though apparently with a small preference for more weight on the bigram probability, which makes sense.

From the basis given here, you can try playing around with some of the other corpora in NLTK and see if you get similar results. You could also implement a trigram model, or another kind of smoothing, to see if you can get better perplexity scores.